In [1]:
import numpy as np
import pandas as pd 
from datetime import datetime
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [2]:
full_data = pd.read_csv('saffronart.csv', encoding = "ISO-8859-1")
full_data.dropna(thresh=1, inplace = True) #drop blank rows
full_data.tail(3)

,artist name,birth date,birth place,death date,death place,education,title,winning bid,low est,high est,auction name,auction date,category,style,provenance,exhibition,details
25062,Nasreen Mohamedi,1937,Karachi India (now in Pakistan),1990,Kihim India,"1961-63 Monsieur Guillard's Atelier, Paris\r\...",Untitled,"11,352\r","8,700","10,870\r",AUTUMN AUCTION 2011,21-22 SEPTEMBER 2011,Drawing,Abstract,"PROVENANCE: | Chatterjee & Lal, Mumbai | Acqui...",NaN,Ink and pencil on card paper | 10.5 x 13.5 in ...
25064,Baiju Parthan,1956,"Kottayam, Kerala",NaN,NaN,"2007 Master in Philosophy, University of Mumb...",Process - (Fruit),"4,426","2,460","4,100\r",KOCHI MUZIRIS BIENNALE FUNDRAISER AUCTION,7-Apr-15,Print Making,Still Life,NaN,NaN,Signed and dated in English (lower right) | 20...
25066,Thota Vaikuntam,1942,Boorugupali Karimnagar Andhra Pradesh,NaN,NaN,"1971-72 Painting and Printmaking, Faculty of F...",Untitled,"26,554\r","11,115","13,335\r",AUTUMN AUCTION 2010,8-9 SEPTEMBER 2010,Painting,Figurative,NaN,NaN,Signed and dated in Telugu (lower left) | 2010...


In [3]:
columns_to_include = ['artist name', 'birth date', 'birth place', 'death date', 
                      'title', 'winning bid', 'auction date', 'category', 'style']

basic_info = full_data.loc[:, columns_to_include].copy()

#rename the column 'artist name' to 'artist':
basic_info.rename(columns = {'artist name': 'artist'}, inplace = True)

#reformat the column 'winning bid':
basic_info['winning bid'] = basic_info['winning bid'].str.lstrip("$").str.rstrip("\r").str.replace(",", "")

#convert string into numeric type:
basic_info['winning bid'] = pd.to_numeric(basic_info['winning bid'])

#reformat artist, category, and style:
for col in ["artist", "category", "style"]:
    basic_info[col] = basic_info[col].str.split().str.join(' ')
    basic_info[col] = basic_info[col].str.lower()

basic_info.head(5)

,artist,birth date,birth place,death date,title,winning bid,auction date,category,style
0,akhilesh,1956,Indore,NaN,Divine Conversation,9000.0,12-13 JUNE 2019,painting,abstract
2,akhilesh,1956,Indore,NaN,In search of Untitled forms -II,1120.0,12-13 NOVEMBER 2013,painting,abstract
4,akhilesh,1956,Indore,NaN,In Search of Untitled Lines,6186.0,7-8 AUGUST 2013,painting,NaN
6,akhilesh,1956,Indore,NaN,Magadhi,2400.0,27-28 FEBRUARY 2013,painting,abstract
8,akhilesh,1956,Indore,NaN,Untitled,2942.0,27-28 FEBRUARY 2013,painting,abstract


In [4]:
def trans_full_form(date_text, formats_list, index):
    #transform date string in full formats like 'june-2019' or '2019'
    
    if index == len(formats_list): #if all given formats were tried
        raise ValueError #not transformable 
    
    try:
        return datetime.strptime(date_text, formats_list[index])
    
    except ValueError: #if wrong format, try next format
        return trans_full_form(date_text, formats_list, index+1)
    
    except TypeError: #if nan, just pass
        return 
    
    
def trans_short_form(date_text, formats_list, index, lowerbound, upperbound):
    #transform date string in short formats like '7-apr-55'
    
    if index == len(formats_list):
        raise ValueError #not transformable
    
    try:
        date = datetime.strptime(date_text, formats_list[index])
    
    except ValueError: #if wrong format, try next format
        date = trans_short_form(date_text, formats_list, index+1, lowerbound, upperbound)
    
    except TypeError: #if nan, just pass
        return
         
    #because it defaults to the latest date, we need to check the lower and upper bounds
    #for example, '7-apr-55' is understood as April 7, 2055 instead of 1955
    
    if date > upperbound:
        return date.replace(year = date.year - 100) #subtract 100 years
    
    elif date < lowerbound:
        return date.replace(year = date.year + 100) #add 100 years

    return date

In [5]:
#convert the column auction date (str) into date:

basic_info['auction date'] = basic_info['auction date'].str.split().str[-2:]
basic_info['auction date'] = basic_info['auction date'].str.join('-').str.lower()

first_auction = datetime(2000, 1, 1)
latest_auction = datetime(2019, 7, 1)

for i, date in enumerate(basic_info['auction date']):
    try: #full format
        basic_info.iat[i, 6] = trans_full_form(date, ['%B-%Y'], 0)
    except ValueError: #try short format: 
        basic_info.iat[i, 6] = trans_short_form(date, ['%d-%b-%y'], 0, first_auction, latest_auction)

In [6]:
#convert the column birth date (str) into date:

first_birth = datetime(1769, 1, 1)
latest_birth = datetime(1988, 1, 1)

for i, date in enumerate(basic_info['birth date']):
    try: #full format
        basic_info.iat[i, 1] = trans_full_form(date, ['%Y', '%B %d, %Y'], 0)
    except ValueError: #try short format: 
        basic_info.iat[i, 1] = trans_short_form(date, ['%d-%b-%y'], 0, first_birth, latest_birth)

In [7]:
#convert the column death date (str) into date:

first_death = datetime(1837, 1, 1)
latest_death = datetime(2019, 3, 21)

for i, date in enumerate(basic_info['death date']):
    try: #full format
        basic_info.iat[i, 3] = trans_full_form(date, ['%Y', '%B %d, %Y'], 0)
    except ValueError: #try short format: 
        basic_info.iat[i, 3] = trans_short_form(date, ['%d-%b-%y'], 0, first_death, latest_death)

In [8]:
def get_age(death_date, birth_date):
    return (death_date - birth_date).days//364


def check_age(age, death_date, birth_date):    
    oldest = 101.0
    youngest = 24.0
    
    if age > oldest:
        death_date = death_date.replace(year = death_date.year - 100) #subtract 100 years
        age = age - 100
    
    elif age < youngest:
        birth_date = birth_date.replace(year = birth_date.year + 100) #add 100 years
        age = age + 100
        
    return age, death_date, birth_date

In [9]:
basic_info['age'] = np.nan
now = datetime(2019, 7, 1) #the date the data was collected

for i in range(len(basic_info.index)):
    try:
        age = get_age(basic_info.iat[i, 3], basic_info.iat[i, 1]) #age = get_age(death date, birth date)
        
        age, death_date, birth_date = check_age(age, basic_info.iat[i, 3], basic_info.iat[i, 1]) #check_age()
        
        basic_info.iat[i, 9] = age
        basic_info.iat[i, 3] = death_date
        basic_info.iat[i, 1] = birth_date
        
    except TypeError: 
        try: #if death date is NaN
            basic_info.iat[i, 9] = get_age(now, basic_info.iat[i, 1]) #age = get_age(death date, birth date)
            
        except TypeError: #if birth date is also NaN
            pass

In [10]:
values = {'style': 'unknown', 'category': 'unknown'} 
basic_info.fillna(value = values, inplace = True) #set NaN values of columns 'style' and 'category' to 'unknown'

In [11]:
basic_info.to_csv("saffronart_basic_info.csv", index = False)